In [14]:
from database import Database
from table import Table
from attribute import Attribute
from database import Database
from TableOperators import *

In [15]:
def create_tables():
    table_name = input("Please input new table name:")
    attrs = []
    print("Please input at least one attribute. Input 'quit' at any time to stop.")

    while True:
        input_str = input("Please input the name and type (use , as delimiter) for one attribute:")
        if input_str == "quit":
            if len(attrs) == 0:
                print("You need to input at least 1 attribute for the new table")
            else:
                break
        else:
            input_split = input_str.replace(" ","").split(",")
            if len(input_split) != 2 or input_split[1] == "":
                print("The input is not correct, please input again.")
            else:
                attrs.append(Attribute(input_split[0], input_split[1]))
                print("Successfully add a new attribute")

    new_table = Table(table_name, attrs)
    print("The tabel " + table_name + " have created successfully!")
    print(new_table)

    return new_table


def create_constraint(table, db):
    while True:
        input_str = input("Please input Boolean condition:")
        if input_str == "quit":
            break
        feedback = table.add_boolean_conditions(input_str.replace(" ",""))
        print(feedback)

    while True:

        input_str = input("Please input FD:")
        if input_str == "quit":
            break
        feedback = table.add_fd(input_str)
        print(feedback)

    while True:
        if len(table.fds) == 0:
            break
        else:
            table.print_fds()
        input_str = input("Do you want to remove some Fds(yes or no):")
        if input_str == "no":
            break
        else:
            input_str = input("Please input a Fd that you want to remove:")
            feedback = table.remove_fd(input_str)
            print(feedback)

    while True:

        input_str = input("Please input MVD:")
        if input_str == "quit":
            break

        feedback = table.add_mvd(input_str)
        print(feedback)

#     while True:

#         input_str = input("Please input foreign key and its table(use , as delimiter):")
#         if input_str == "quit":
#             break
#         input_split = input_str.replace(" ","").split(",")
#         if len(input_split) != 2 or input_split[1] == "":
#             print("Invalid input, please input again!")
#             continue
#         if input_split[1] not in db.tables:
#             print("There is no such table!")
#             continue
#         else:
#             foregin_table = db.tables[input_split[1]]

#         result = foregin_table.get_keys().pop()
#         if not input_split[0] in result:
#             print("The key is not the key in the table " + input_split[1] + ".")
#             continue

#         table.add_foreign_key(input_str[0], foregin_table)
#         print("Add foreign key successfully")

    return table


def update_normal_form(db, table):
    if table.nf == "":
        table.get_normal_form()

    print("Currently this table [" + table.name + "] has normal form: " + table.nf)

    if not (table.nf == "3NF" or table.nf == "BCNF"):
        print("You can either delete this table or update its FD's.")
        decision = input("Either type \'del\' for deletion of the table or \'u\' to update:")

        if decision == 'del':
            input_str = input("Are you sure delete the table "+ table.name +"?(yea or no)")
            if input_str == "yes":
                db.delete_table(table.name)
                print("delete the tabel [" + table.name + "] successfuly")
                table = None
            else:
                update_normal_form(db, table)
        elif decision == 'u':
            done = False
            while not done:
                print("Current FD(s) in the table: " + str(table.fds) + "\n")
                input_str = input("Type \'a\' to add a new FD or \'d\' to delete a FD: ")
                if input_str == 'a':
                    new_fd = input("Add a new FD: ")
                    feedback = table.add_fd(new_fd.replace(" ",""))
                    print(feedback)
                    table.get_normal_form()
                    again = input("The table now has normal form " + table.nf + " would you like to continue (y/n)?")
                    if again == 'n':
                        done = True

                elif input_str == 'd':
                    del_fd = input("Enter FD to be deleted: ")
                    feedback = table.remove_fd(del_fd.replace(" ",""))
                    print(feedback)
                    if len(table.fds) == 0:
                        print("The table has no FD left! Please start again")
                        break

                    else:
                        table.get_normal_form()
                        again = input("The table now has normal form " + table.nf + " would you like to continue (y/n)?")
                        if again == 'n':
                            done = True
                else:
                    print("That's not a valid decision!")

        else:
            print("That's not a valid decision!")

    return table

def user_define_key(table):
    table.get_keys()
    print("Please define keys for the table [" + table.name + "].")
    table.user_define_key()


In [17]:
db = Database()
# Task I.a
# Define new tables
while True:
    new_table = create_tables()
    db.add_table(new_table)
    is_add_new_table = input("Do you still want to define a new table(yes or no)?")
    if is_add_new_table == "no":
        break

Please input new table name:Student
Please input at least one attribute. Input 'quit' at any time to stop.
Please input the name and type (use , as delimiter) for one attribute:A, int
Successfully add a new attribute
Please input the name and type (use , as delimiter) for one attribute:B, int
Successfully add a new attribute
Please input the name and type (use , as delimiter) for one attribute:C, int
Successfully add a new attribute
Please input the name and type (use , as delimiter) for one attribute:quit
The tabel Student have created successfully!
Table: Student
 | A | B | C |
---------------
Do you still want to define a new table(yes or no)?yes
Please input new table name:Course
Please input at least one attribute. Input 'quit' at any time to stop.
Please input the name and type (use , as delimiter) for one attribute:A, int
Successfully add a new attribute
Please input the name and type (use , as delimiter) for one attribute:D, str
Successfully add a new attribute
Please input the

In [18]:
# Task I.b
# Ask users to input possible constraints for each defined table
for table_name, table in db.tables.items():
    print("Please input constraint for the table [" + table.name + "]. Input 'quit' at any time to stop.")
    table = create_constraint(table, db)

Please input constraint for the table [Student]. Input 'quit' at any time to stop.
Please input Boolean condition:quit
Please input FD:A->B
Added a new fd successfully: A->B
Please input FD:quit
The table named:  Student 
Has FD's:  ['A->B']
Do you want to remove some Fds(yes or no):no
Please input MVD:quit
Please input constraint for the table [Course]. Input 'quit' at any time to stop.
Please input Boolean condition:A->D
There is no the attribute in the table
Please input Boolean condition:quit
Please input FD:A->D
Added a new fd successfully: A->D
Please input FD:quit
The table named:  Course 
Has FD's:  ['A->D']
Do you want to remove some Fds(yes or no):no
Please input MVD:quit


In [19]:
# Task I.c
# Evaluate the NF category for the user defined DB
for table_name, table in db.tables.copy().items():
    table = update_normal_form(db, table)
    if table != None:
        db.tables[table_name] = table

This table has normal form: 1NF
Currently this table [Student] has normal form: 1NF
You can either delete this table or update its FD's.
Either type 'del' for deletion of the table or 'u' to update:u
Current FD(s) in the table: ['A->B']

Type 'a' to add a new FD or 'd' to delete a FD: a
Add a new FD: A->C
Added a new fd successfully: A->C
This table has normal form: BCNF
The table now has normal form BCNF would you like to continue (y/n)?y
Current FD(s) in the table: ['A->B', 'A->C']

Type 'a' to add a new FD or 'd' to delete a FD: quit
That's not a valid decision!
Current FD(s) in the table: ['A->B', 'A->C']

Type 'a' to add a new FD or 'd' to delete a FD: quit
That's not a valid decision!
Current FD(s) in the table: ['A->B', 'A->C']

Type 'a' to add a new FD or 'd' to delete a FD: a
Add a new FD: quit
Must have a LHS & RHS!
This table has normal form: BCNF
The table now has normal form BCNF would you like to continue (y/n)?n
This table has normal form: 1NF
Currently this table [Cours

In [ ]:
# Task I.d
# Ask users to define keys for each table
for table_name, table in db.tables.items():
    user_define_key(table)

In [21]:
 # Task II.a
    # Input new tuples to all tables
    # db = fake_data()
    while True:
        print("The tables in current db: \n", db)
        input_str = input("Please choose a table in order to input a new tuple into it(or quit to terminate): ")
        if input_str == 'quit':
            break
        if input_str in db.tables:
            table = db.tables[input_str]
            while True:
                input_tuple = input("Please input a new tuple into the table [" + input_str + "]:")
                if input_tuple == 'quit':
                    break
                table.add_tuple(eval(input_tuple))
        else:
            print("Invaild input, please input again!")

The tables in current db: 
 Table: Student
 | B | C |
---------------Table: Course
 | D | E |
---------------
Please choose a table in order to input a new tuple into it(or quit to terminate): Student
Please input a new tuple into the table [Student]:1 2 3


SyntaxError: invalid syntax (<string>, line 1)

In [ ]:
# Task II.b
    # Delete a tuple based on key value
    while True:
        print("The tables in current db: \n", db)
        input_str = input("Would you like to delete a tuple from any table? (y/n)")
        if input_str == 'n':
            break
        else:
            table_name = input("Which table would you like to delete from?")
            table = db.tables[table_name]
            print("This the table you will be deleting from: \n", table)
            print("Which has the key: ", table.keys)
            key = input("Please provide the key for the tuple you'd like to delete: ")
            table.remove_tuple(key)


In [ ]:
# Task II.c
# TODO: find tuples based on conditions

# Group by attribute(s) in specific table
while True:
    print("The tables in current db: \n", db)
    input_str = input("Would you like to group by any attributes from the tables (y/n)?")
    if input_str == 'n':
        break
    else:
        table_name = input("Which table would you like to group by in?")
        table = db.tables[table_name]
        attr_name = input("Which attribute would you like to group by?")
        table.group_by(attr_name)

In [23]:
# TODO Task II.d
# Choose to perform the operators for two tables
while True:
    print("The tables in current db: \n", db)
    input_str = input("Would you like to perform operators on tables (y/n)?")
    if input_str == 'n':
        break
    else:
        table_one = input("Select the first table: ")
        table1 = db.tables[table_one]
        table_two = input("Select the second table: ")
        table2 = db.tables[table_two]
        while True:
            input_str = input('Select operators:\
            \n \'cjoin\' for cross join\
                              \n \'njoin\' for natural join\
                              \n \'union\' for union\
                              \n \'inter\' for intersection\
                              \n \'diff\' for difference\n')
            if input_str == 'njoin':
                njoin_table(table1,table2)
            if input_str == 'cjoin':
                cjoin_table(table1,table2)
            if input_str == 'union':
                union(table1,table2)
            if input_str == 'inter':
                intersection(table1,table2)
            if input_str == 'diff':
                difference(table1,table2)
            else:
                break

The tables in current db: 
 Table: Student
 | B | C |
---------------Table: Course
 | D | E |
---------------
Would you like to perform operators on tables (y/n)?y
Select the first table: Student
Select the second table: Course
Select operators:            
 'cjoin' for cross join                              
 'njoin' for natural join                              
 'union' for union                              
 'inter' for intersection                              
 'diff' for difference
cjoin
The tables in current db: 
 Table: Student
 | B | C |
---------------Table: Course
 | D | E |
---------------
Would you like to perform operators on tables (y/n)?y
Select the first table: Student
Select the second table: Course
Select operators:            
 'cjoin' for cross join                              
 'njoin' for natural join                              
 'union' for union                              
 'inter' for intersection                              
 'diff' for difference
di

KeyError: 'B'

In [19]:
# TODO Task II.e
# Delete a table and ensure the across-table integrity